## **Covid\_in\_numbers** 

_an SQL project_

In [ ]:
SELECT * 
FROM [Portfolio Project]..CovidDeaths
WHERE continent is not null
ORDER BY 3,4

In [ ]:
SELECT * 
FROM [Portfolio Project]..CovidVaccinations
ORDER BY 3,4

Selecting the data that we are going to be using

In [ ]:
SELECT Location, Date, total_cases, new_cases, total_deaths, population
FROM [Portfolio Project]..CovidDeaths
WHERE continent is not null
ORDER BY 1,2

Total Cases vs. Total Deaths

likelihood of dying if covid positve in Croatia

In [ ]:
SELECT Location, Date, total_cases, total_deaths, (total_deaths/total_cases)*100 as DeathPercentage
FROM [Portfolio Project]..CovidDeaths
WHERE [location] LIKE 'Croatia'
ORDER BY 1,2

Total Cases vs Population

Percentage of population infected by Covid

In [ ]:
SELECT Location, Date, total_cases, population, (total_cases/population)*100 as CovidPercentage
FROM [Portfolio Project]..CovidDeaths
WHERE [location] LIKE 'Croatia'
ORDER BY 1,2

 Countries with highest infection rate compared to population

In [ ]:
SELECT Location, Max(total_cases) as HighestInfectionCount, MAX((total_cases/population)*100) as PercentOfPopulationInfected
FROM [Portfolio Project]..CovidDeaths
-- WHERE [location] LIKE 'Croatia'
WHERE continent is not null
GROUP BY [location],population 
ORDER BY PercentOfPopulationInfected DESC

 Counties with the highest Death Count per Population

In [ ]:
SELECT Location, Max(total_deaths) as TotalDeathCount
FROM [Portfolio Project]..CovidDeaths
-- WHERE [location] LIKE 'Croatia'
WHERE continent is not null
GROUP BY [location]
ORDER BY TotalDeathCount DESC

Continent with total death count

In [ ]:
SELECT [location], Max(total_deaths) as TotalDeathCount
FROM [Portfolio Project]..CovidDeaths
-- WHERE [location] LIKE 'Croatia'
WHERE continent is null 
and [location] not in ('High income','Upper middle income', 'Lower middle income','Low income')
GROUP BY [location] 
ORDER BY TotalDeathCount DESC

Global numbers: Total Cases vs Total Deaths with the daily percent

In [ ]:
SELECT Date, SUM(new_cases) as total_cases, SUM(new_deaths) as total_deaths
, SUM(cast(new_deaths as float))/nullif(SUM(cast(new_cases as float)),0)*100 as DeathPercentage
FROM [Portfolio Project]..CovidDeaths
--WHERE [location] LIKE 'Croatia'
WHERE continent is not null
GROUP BY [date]
ORDER BY 1,2

Overall Total Cases vs Deaths and Death Pecentage

In [ ]:
SELECT SUM(new_cases) as total_cases, SUM(new_deaths) as total_deaths
, SUM(cast(new_deaths as float))/nullif(SUM(cast(new_cases as float)),0)*100 as DeathPercentage
FROM [Portfolio Project]..CovidDeaths
--WHERE [location] LIKE 'Croatia'
WHERE continent is not null
--GROUP BY [date]
ORDER BY 1,2

Joining two tables, 

Total Population vs Vaccination, with total vacciantions per day per country

In [ ]:
SELECT dea.continent, dea.[location],dea.[date], dea.population, vac.new_vaccinations
, SUM(vac.new_vaccinations) OVER (Partition by dea.location Order by dea.location, dea.[date]) as RollingPeopleVaccinated
FROM [Portfolio Project]..CovidDeaths dea 
JOIN [Portfolio Project]..CovidVaccinations vac 
    ON dea.[location] = vac.[location]
    and dea.[date] = vac.[date]
WHERE dea.continent is not null and vac.new_vaccinations is not null
ORDER BY continent,[location]


Population vs Vaccination. //  with CTE

In [ ]:
with PopvsVac (continent,location,date,population, New_Vaccinations, RollingPeopleVaccinated)
as
(
SELECT dea.continent, dea.[location],dea.[date], dea.population, vac.new_vaccinations
, SUM(vac.new_vaccinations) OVER (Partition by dea.location Order by dea.location, dea.[date]) as RollingPeopleVaccinated
FROM [Portfolio Project]..CovidDeaths dea 
JOIN [Portfolio Project]..CovidVaccinations vac 
    ON dea.[location] = vac.[location]
    and dea.[date] = vac.[date]
WHERE dea.continent is not null and vac.new_vaccinations is not null
--ORDER BY continent,[location]
)
SELECT *, (RollingPeopleVaccinated/population)*100 RollingPercentVaccinated
FROM PopvsVac

Populatin vs vaccination // with temp table

In [ ]:
DROP TABLE if EXISTS #percentPopulationVaccinated
CREATE TABLE #percentPopulationVaccinated
(

Continent NVARCHAR(255),
Location NVARCHAR(255),
Date datetime,
Population numeric, 
New_vaccinations numeric,
RollingPeopleVaccinated numeric
)

INSERT into #percentPopulationVaccinated
SELECT dea.continent, dea.[location],dea.[date], dea.population, vac.new_vaccinations
, SUM(vac.new_vaccinations) OVER (Partition by dea.location Order by dea.location, dea.[date]) as RollingPeopleVaccinated
FROM [Portfolio Project]..CovidDeaths dea 
JOIN [Portfolio Project]..CovidVaccinations vac 
    ON dea.[location] = vac.[location]
    and dea.[date] = vac.[date]
WHERE dea.continent is not null and vac.new_vaccinations is not null
--ORDER BY continent,[location]

SELECT *, (RollingPeopleVaccinated/population)*100 RollingPercentVaccinated
FROM #percentPopulationVaccinated
ORDER BY location

Creating Views for later visualization

In [ ]:
CREATE VIEW PercentPopulationVaccinated as 
SELECT dea.continent, dea.[location],dea.[date], dea.population, vac.new_vaccinations
, SUM(vac.new_vaccinations) OVER (Partition by dea.location Order by dea.location, dea.[date]) as RollingPeopleVaccinated
FROM [Portfolio Project]..CovidDeaths dea 
JOIN [Portfolio Project]..CovidVaccinations vac 
    ON dea.[location] = vac.[location]
    and dea.[date] = vac.[date]
WHERE dea.continent is not null
--ORDER BY continent,[location]

In [ ]:
Select * 
FROM PercentPopulationVaccinated
ORDER By 2, 3 ASC